# Semianr 6 - Applied Quantitative Logistics

## Particle Swarm Optimization (PSO) - Binary

In [ ]:
import math
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt

### HubLocation Allocation

In [ ]:
def hubLocation():
    
    # Customer information
    xc = [94, 60, 62, 3, 23, 43, 27, 62, 97, 75, 54,
          56, 58, 0, 62, 28, 27, 86, 7, 52, 65, 0, 32,
          15, 3, 61, 16, 65, 63, 43, 67, 53, 98, 68,
          34, 32, 26, 18, 33, 66]
    
    yc = [67, 33, 11, 2, 48, 88, 17, 71, 38, 78,
          37, 72, 58, 12, 54, 5, 24, 76, 38, 61,
          82, 65, 66, 55, 17, 79, 72, 14, 45, 30,
          36, 97, 15, 52, 30, 66, 13, 42, 77, 12]
    
    d = [13, 44, 12, 30, 5, 44, 18, 29, 20, 6, 35,
         16, 17, 31, 32, 9, 37, 30, 8, 21, 31, 17,
         47, 11, 6, 14, 40, 48, 31, 16, 32, 14, 44,
         42, 6, 26, 20, 43, 14, 20]
    
    N = len(xc)
    
    # Service center information
    xs = [63, 91, 0, 54, 39, 55, 15, 50, 21, 92,
          12, 81, 41, 5, 62, 75, 62, 20, 77, 57]
    
    ys = [37, 25, 37, 12, 43, 57, 98, 2, 85, 87,
          0, 75, 30, 30, 85, 14, 65, 78, 95, 36]
    
    M = len(xs)
    
    # calculate the distance
    D = np.zeros([N, M])
    
    for i in range(N):
        for j in range(M):
            D[i][j] = abs(xc[i]-xs[j]) + abs(yc[i]-ys[j])
    
    model = {'N': N,
             'M': M,
             'xc': xc,
             'yc': yc,
             'xs': xs,
             'ys': ys,
             'd': d,
             'D': D}
    
    return model

### Create Random Solution

In [ ]:
def createRandomSolution(model):
    M = model['M']
    f = list(np.random.randint(0, 2, M))
    
    return f

### Cost Function

In [ ]:
def myCost(f, model):
    
    global NFE
    
    if pd.isna(NFE):
        NFE = 0

    NFE += 1
    
    # If no center is activated
    # The cost of the system is inf
    if (np.all(np.array(f) == 0)):
        z = math.inf
        return z

    N = model['N']
    M = model['M']
    D = model['D']
    
    D_min = np.zeros(N)
    
    for i in range(N):
        D_temp = []
        for j in range(M):
            if f[j] == 1:
                D_temp.append(D[i][j])
                
        D_min[i] = min(D_temp)
        
    z = sum(np.array(model['d']) * np.array(D_min))
    
    return z

### Sort Population

In [ ]:
# Sort the population and cost (based on the cost)
def pop_sort(p, c):
    li = []
    for i in range(len(c)):
        li.append([c[i],i])
        
    li.sort()
    sort_index = []
    
    for x in li:
        sort_index.append(x[1])
    
    positions, cost = [], []
    for i in sort_index:
        positions.append(p[i])
        cost.append(c[i])
        
    return positions, cost

### PSO Algorithm - Binary

In [ ]:
### Problem Definition
model = hubLocation()

nPar = model['M']

global NFE
NFE = 0

nfe = []

# PSO parameters
swarmSize = 40
maxIter = 100

C1 = 2
C2 = 4-C1

Position, Costs, Velocity, LBPosition, LBCost = [], [], [], [], [] 

# Initialize swarm

GBPosition = createRandomSolution(model)
GBCost = math.inf

GBCost_list = []

for i in range(swarmSize):
    Position.append(createRandomSolution(model))
    Velocity.append(list(np.random.random_sample(nPar)))
    Costs.append(myCost(Position[i], model))
    
    LBPosition.append(Position[i])
    LBCost.append(Costs[i])
    
    if LBCost[i] < GBCost:
        GBPosition = LBPosition[i]
        GBCost = LBCost[i]

# Main body for PSO
for it in range(maxIter):
    
    for ii in range(swarmSize):
        # Velocity Update
        Velocity[ii] = list(C1*np.random.rand()*(np.array(LBPosition[ii])-np.array(Position[ii])) \
                           + C2*np.random.rand()*(np.array(GBPosition)-np.array(Position[ii])))
        
        # Position Update
        SigV = 1/(1+np.exp(-np.array(Velocity[ii])))
        my_rand = np.random.random_sample(nPar)
        Position[ii] = [int(SigV[k] > my_rand[k]) for k in range(len(SigV))]
        Costs[ii] = myCost(Position[ii], model)
    
        # Global and Local Update
        if Costs[ii] < LBCost[ii]:
            LBPosition[ii] = Position[ii]
            LBCost[ii] = Costs[ii]
        
        if LBCost[ii] < GBCost:
            GBPosition = LBPosition[ii]
            GBCost = LBCost[ii]
            
    
    GBCost_list.append(GBCost)
    
    # Append NFE to the nfe list
    nfe.append(NFE)
    
    print(f'Iteration {it} : NFE = {nfe[-1]}, Best Cost = {GBCost_list[it]}')

In [ ]:
# plot the result
plt.plot(GBCost_list, linewidth=3)
plt.xlabel('Iteration')
plt.ylabel('Costs')